# Finding the best option to invest in Warsaw Venues

Setup and Libraries import

In [1]:
!pip install geocoder
!pip install folium

    100% |████████████████████████████████| 102kB 14.4MB/s a 0:00:01
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
    100% |████████████████████████████████| 92kB 14.1MB/s ta 0:00:01
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import geocoder
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

Getting the data from Wikipedia

In [3]:
wiki_link = 'https://pl.wikipedia.org/wiki/Warszawa'
page = requests.get(wiki_link).text

In [4]:
soup = BeautifulSoup(page, 'lxml')
my_table = soup.find('table',{'class':'wikitable sortable'})

In [5]:
data =[]
rows = my_table.find_all('tr')

for tr in rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    if row:
        data.append(row)

In [6]:
data

[['Mokotów', '217 577', '6143', '35,42\n'],
 ['Praga-Południe', '178 726', '7986', '22,38\n'],
 ['Ursynów', '150 273', '3432', '43,79\n'],
 ['Wola', '140 111', '7275', '19,26\n'],
 ['Bielany', '132 026', '4082', '32,34\n'],
 ['Targówek', '123 941', '5117', '24,22\n'],
 ['Bemowo', '122 210', '4898', '24,95\n'],
 ['Śródmieście', '117 005', '7515', '15,57\n'],
 ['Białołęka', '119 374', '1634', '73,04\n'],
 ['Ochota', '83 081', '8547', '9,72\n'],
 ['Wawer', '75 991', '991', '79,70\n'],
 ['Praga-Północ', '64 904', '5683', '11,42\n'],
 ['Ursus', '59 261', '6331', '9,36\n'],
 ['Żoliborz', '51 441', '6073', '8,47\n'],
 ['Włochy', '41 929', '1465', '28,63\n'],
 ['Wilanów', '35 511', '1021', '36,73\n'],
 ['Wesoła', '25 106', '1094', '22,94\n'],
 ['Rembertów', '24 148', '1251', '19,30\n']]

In [7]:
df = pd.DataFrame(data, columns = ['Borough', 'Inhabitants', 'Density', 'Area'])
df = df[df.Borough !='Not assigned']
df['Area'].replace(r"\n","", regex=True, inplace=True)
df['Area'].replace(r"\,",".", regex=True, inplace=True)
df['Inhabitants'].replace(r"\ ","", regex=True, inplace=True)
df['Inhabitants'] = df['Inhabitants'].astype('int')
#df['']
#df['Neighbourhood'].replace(to_replace = 'Not assigned', value = df['Borough'], inplace = True)
df.head()

,Borough,Inhabitants,Density,Area
0,Mokotów,217577,6143,35.42
1,Praga-Południe,178726,7986,22.38
2,Ursynów,150273,3432,43.79
3,Wola,140111,7275,19.26
4,Bielany,132026,4082,32.34


In [8]:
!pip install geopy

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


Geolocating the data

In [9]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")



In [10]:
#for B in df['Borough']:
 #   location = geolocator.geocode(B)
 #   df_geo = pd.DataFrame(B, location.latitude, location.longitude)

df_geo = pd.DataFrame(columns=('Borough', 'lat', 'long'))
for B in df['Borough']:
    city = 'Warsaw'
    x1 = str((B,city))
    location = geolocator.geocode(x1)
    df_geo.loc[B] = [B,location.latitude, location.longitude]
    
df_geo.head(20)

,Borough,lat,long
Mokotów,Mokotów,52.193987,21.045781
Praga-Południe,Praga-Południe,52.237396,21.071258
Ursynów,Ursynów,52.141039,21.032321
Wola,Wola,52.236238,20.954781
Bielany,Bielany,52.294652,20.929980
Targówek,Targówek,52.275192,21.058085
Bemowo,Bemowo,52.238974,20.913288
Śródmieście,Śródmieście,52.232810,21.019067
Białołęka,Białołęka,52.319665,21.021177
Ochota,Ochota,52.212225,20.972630


In [11]:
df_war = pd.merge(df, df_geo, on='Borough')
df_war.head()

,Borough,Inhabitants,Density,Area,lat,long
0,Mokotów,217577,6143,35.42,52.193987,21.045781
1,Praga-Południe,178726,7986,22.38,52.237396,21.071258
2,Ursynów,150273,3432,43.79,52.141039,21.032321
3,Wola,140111,7275,19.26,52.236238,20.954781
4,Bielany,132026,4082,32.34,52.294652,20.929980


In [12]:
address = 'Warsaw'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_war = folium.Map(location=[latitude, longitude], zoom_start=11)

# markers_colors = []
for lat, long, Borough, size in zip(df_war["lat"], df_war["long"], df_war["Borough"],  df_war["Inhabitants"]):
    label = '{}, {}'.format(Borough, size)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        label = Borough,
        radius=(size/5000),
        color='Blue',
        fill=True).add_to(map_war)
map_war


Importing 4square data about venues

In [13]:
CLIENT_ID = 'XOYO3TBFES0XHPQVOMO3C5HHUVLITT3RLESAYOXUVU2YDMHE' # your Foursquare ID
CLIENT_SECRET = 'NOFGL55ANZYBXW3ON3NGJEXEA1FWZR3EZZ2KT3SKQO4YTVXW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XOYO3TBFES0XHPQVOMO3C5HHUVLITT3RLESAYOXUVU2YDMHE
CLIENT_SECRET:NOFGL55ANZYBXW3ON3NGJEXEA1FWZR3EZZ2KT3SKQO4YTVXW


In [14]:
# type your answer here
radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=XOYO3TBFES0XHPQVOMO3C5HHUVLITT3RLESAYOXUVU2YDMHE&client_secret=NOFGL55ANZYBXW3ON3NGJEXEA1FWZR3EZZ2KT3SKQO4YTVXW&v=20180605&ll=52.2319237,21.0067265&radius=500&limit=100'

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
results = requests.get(url).json()
results

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Pałac Kultury i Nauki,Building,52.231906,21.007064
1,Bar Studio,Cocktail Bar,52.232452,21.006705
2,Kinoteka,Movie Theater,52.231087,21.006663
3,Teatr Dramatyczny,Theater,52.231435,21.007551
4,RiverView Wellness Centre,Hotel Pool,52.232198,21.002454


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:

war_venues = getNearbyVenues(names=df_war['Borough'],
                                   latitudes=df_war['lat'],
                                   longitudes=df_war['long']
                                  )


Mokotów
Praga-Południe
Ursynów
Wola
Bielany
Targówek
Bemowo
Śródmieście
Białołęka
Ochota
Wawer
Praga-Północ
Ursus
Żoliborz
Włochy
Wilanów
Wesoła
Rembertów


In [19]:
print(war_venues.shape)
war_venues.head()

(1073, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Mokotów,52.193987,21.045781,Tor Łyżwiarski Stegny,52.187430,21.042964,Skating Rink
1,Mokotów,52.193987,21.045781,Park Królikarnia,52.189391,21.027167,Park
2,Mokotów,52.193987,21.045781,Centrum Futbolu Warszawianka,52.195873,21.028518,Soccer Field
3,Mokotów,52.193987,21.045781,Park Arkadia,52.190851,21.028433,Park
4,Mokotów,52.193987,21.045781,Warszawianka Korty,52.197041,21.030077,Tennis Court


In [20]:
war_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bemowo,48,48,48,48,48,48
Białołęka,12,12,12,12,12,12
Bielany,43,43,43,43,43,43
Mokotów,98,98,98,98,98,98
Ochota,100,100,100,100,100,100
Praga-Południe,100,100,100,100,100,100
Praga-Północ,100,100,100,100,100,100
Rembertów,8,8,8,8,8,8
Targówek,33,33,33,33,33,33


In [21]:
# one hot encoding
war_onehot = pd.get_dummies(war_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
war_onehot.insert(0, "Borough", war_venues['Neighborhood'], True) 
#war_onehot["Neighborhood"] = war_venues["Neighborhood"] 
 

# move neighborhood column to the first column
#fixed_columns = [war_onehot.columns[-1]] + list(war_onehot.columns[:-1])
#war_onehot = war_onehot[fixed_columns]

war_onehot.head()
#print(war_onehot.shape)

,Borough,Airport Lounge,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wine Shop,Yoga Studio,Zoo,Zoo Exhibit
0,Mokotów,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Mokotów,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Mokotów,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Mokotów,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Mokotów,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Analysis - 1st approach - choosing the best venue option in each neighbourhood

### Calculating frequency and venue selection

In [22]:
war_grouped = war_onehot.groupby('Borough').mean().reset_index()
war_grouped

,Borough,Airport Lounge,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wine Shop,Yoga Studio,Zoo,Zoo Exhibit
0,Bemowo,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.00,0.00,0.000000
1,Białołęka,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.083333,0.000000,0.000000,...,0.00,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000
2,Bielany,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.023256,0.000000,...,0.00,0.000000,0.000000,0.023256,0.000000,0.000000,0.000000,0.00,0.00,0.000000
3,Mokotów,0.000000,0.000000,0.00,0.010204,0.000000,0.00,0.020408,0.010204,0.000000,...,0.00,0.010204,0.000000,0.000000,0.000000,0.010204,0.010204,0.00,0.00,0.000000
4,Ochota,0.000000,0.020000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,...,0.01,0.020000,0.000000,0.000000,0.000000,0.010000,0.010000,0.00,0.00,0.000000
5,Praga-Południe,0.000000,0.000000,0.00,0.000000,0.010000,0.00,0.000000,0.010000,0.000000,...,0.00,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.01,0.00,0.000000
6,Praga-Północ,0.000000,0.000000,0.01,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.01,0.030000
7,Rembertów,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000
8,Targówek,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000
9,Ursus,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.00,0.000000,0.022222,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000


### Sorting venues by the most popular in total city

In [109]:
war_grouped_tot = pd.DataFrame(war_onehot.mean(), columns = ['freqtot']).sort_values('freqtot', ascending = False)
war_grouped_tot.reset_index(inplace=True)
war_grouped_tot.columns =  ['venue','freqtot']
war_grouped_tot.head()
#print("The most popular venue in Warsaw is: "+war_grouped_tot.iat[0,0])

,venue,freqtot
0,Café,0.068034
1,Italian Restaurant,0.041007
2,Park,0.041007
3,Supermarket,0.036347
4,Coffee Shop,0.028891


### Now, I will use this hierarchy, and add it to the list of the most popular Venue in each Borough. Total city popularity divided by local popularity will give us the list of potential investment opprtunity in each Neighbourhood

In [119]:
num_top_venues = 1

for hood in war_grouped['Borough']:
    print("----"+hood+"----")
    temp = war_grouped[war_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    tempm = pd.merge(temp, war_grouped_tot, on='venue',how='inner')
    #temp['freqtot'] = war_grouped_tot['freqtot']
    tempm = tempm.iloc[1:]
    #tempm['ratio'] = tempm['freqtot'] / tempm['freq']
    tempm['ratio'] = tempm['freqtot'].div(tempm['freq'].where(tempm['freq'] != 0, np.nan))
    tempm['freq'] = tempm['freq'].astype(float)
    tempm['ratio'] = tempm['ratio'].astype(float)
    tempm = tempm.round({'freq': 2, 'freqtot': 2, 'ratio': 2})

    print(tempm.sort_values('ratio', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Bemowo----
  venue  freq  freqtot  ratio
0  Park  0.02     0.04   1.97


----Białołęka----
         venue  freq  freqtot  ratio
0  Supermarket  0.08     0.04   0.44


----Bielany----
  venue  freq  freqtot  ratio
0  Park  0.02     0.04   1.76


----Mokotów----
         venue  freq  freqtot  ratio
0  Supermarket  0.01     0.04   3.56


----Ochota----
        venue  freq  freqtot  ratio
0  Restaurant  0.01     0.02   2.24


----Praga-Południe----
   venue  freq  freqtot  ratio
0  Hotel  0.01     0.02   2.33


----Praga-Północ----
         venue  freq  freqtot  ratio
0  Supermarket  0.01     0.04   3.63


----Rembertów----
  venue  freq  freqtot  ratio
0  Café  0.12     0.07   0.54


----Targówek----
                  venue  freq  freqtot  ratio
0  Gym / Fitness Center  0.03     0.03   0.95


----Ursus----
         venue  freq  freqtot  ratio
0  Coffee Shop  0.02     0.03    1.3


----Ursynów----
  venue  freq  freqtot  ratio
0  Café  0.02     0.07    3.1


----Wawer----
  venue  freq

## Analysis - 2nd approach - now I will assume that I do want to invest in only one outlet type. I will choose the outlet that is the most popular in total City and select Neighbourhoods where it is underrepresented

In [120]:
print("The most popular venue in Warsaw is: "+war_grouped_tot.iat[0,0])

The most popular venue in Warsaw is: Café


In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = war_grouped['Borough']

for ind in np.arange(war_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(war_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(20)

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bemowo,Supermarket,Café,Dessert Shop,Market,Shopping Mall,Gym / Fitness Center,Italian Restaurant,Gym,Electronics Store,Fast Food Restaurant
1,Białołęka,Flower Shop,Asian Restaurant,Outdoors & Recreation,Rock Climbing Spot,Train Station,Supermarket,Farmers Market,Tennis Court,Vietnamese Restaurant,Gas Station
2,Bielany,Café,Tram Station,Supermarket,Grocery Store,Deli / Bodega,Fast Food Restaurant,Bus Station,Gym / Fitness Center,Gym,Gun Range
3,Mokotów,Park,Café,Gym,Coffee Shop,Burger Joint,Italian Restaurant,Market,Sushi Restaurant,Hotel,Convenience Store
4,Ochota,Café,Park,Italian Restaurant,Pub,Coffee Shop,Sushi Restaurant,Hotel,Gym / Fitness Center,Dance Studio,Vegetarian / Vegan Restaurant
5,Praga-Południe,Café,Restaurant,Italian Restaurant,Supermarket,Park,Ice Cream Shop,Gym / Fitness Center,Coffee Shop,Dessert Shop,Pizza Place
6,Praga-Północ,Café,Plaza,Italian Restaurant,Bar,Polish Restaurant,Zoo Exhibit,Park,Diner,Beer Bar,Restaurant
7,Rembertów,Bus Station,Park,Café,Supermarket,Gun Range,Gym Pool,Fast Food Restaurant,Forest,Food Truck,Food Court
8,Targówek,Supermarket,Plaza,Market,Music Venue,Bus Stop,Pool,Pizza Place,Eastern European Restaurant,Liquor Store,Bus Line
9,Ursus,Supermarket,Italian Restaurant,Hotel,Park,Gym / Fitness Center,Gym Pool,Fast Food Restaurant,Bus Station,Train Station,Gym


In [26]:
#merging df
war_merged = df_war

# merge grouped with data to add latitude/longitude for each neighborhood
war_merged = war_merged.join(neighborhoods_venues_sorted.set_index('Borough'), on='Borough')

war_merged.head(20)

,Borough,Inhabitants,Density,Area,lat,long,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Mokotów,217577,6143,35.42,52.193987,21.045781,Park,Café,Gym,Coffee Shop,Burger Joint,Italian Restaurant,Market,Sushi Restaurant,Hotel,Convenience Store
1,Praga-Południe,178726,7986,22.38,52.237396,21.071258,Café,Restaurant,Italian Restaurant,Supermarket,Park,Ice Cream Shop,Gym / Fitness Center,Coffee Shop,Dessert Shop,Pizza Place
2,Ursynów,150273,3432,43.79,52.141039,21.032321,Convenience Store,Chinese Restaurant,Supermarket,Pizza Place,Italian Restaurant,Indian Restaurant,Burger Joint,Park,Sushi Restaurant,Electronics Store
3,Wola,140111,7275,19.26,52.236238,20.954781,Café,Pizza Place,Park,Grocery Store,Coffee Shop,Supermarket,Gym / Fitness Center,Restaurant,Gym,Sushi Restaurant
4,Bielany,132026,4082,32.34,52.294652,20.929980,Café,Tram Station,Supermarket,Grocery Store,Deli / Bodega,Fast Food Restaurant,Bus Station,Gym / Fitness Center,Gym,Gun Range
5,Targówek,123941,5117,24.22,52.275192,21.058085,Supermarket,Plaza,Market,Music Venue,Bus Stop,Pool,Pizza Place,Eastern European Restaurant,Liquor Store,Bus Line
6,Bemowo,122210,4898,24.95,52.238974,20.913288,Supermarket,Café,Dessert Shop,Market,Shopping Mall,Gym / Fitness Center,Italian Restaurant,Gym,Electronics Store,Fast Food Restaurant
7,Śródmieście,117005,7515,15.57,52.232810,21.019067,Cocktail Bar,Café,Beer Bar,Vegetarian / Vegan Restaurant,Coffee Shop,Boutique,Dessert Shop,Sushi Restaurant,Hotel,Italian Restaurant
8,Białołęka,119374,1634,73.04,52.319665,21.021177,Flower Shop,Asian Restaurant,Outdoors & Recreation,Rock Climbing Spot,Train Station,Supermarket,Farmers Market,Tennis Court,Vietnamese Restaurant,Gas Station
9,Ochota,83081,8547,9.72,52.212225,20.972630,Café,Park,Italian Restaurant,Pub,Coffee Shop,Sushi Restaurant,Hotel,Gym / Fitness Center,Dance Studio,Vegetarian / Vegan Restaurant


## And see howit looks neighbourhood by neighbourhood

In [121]:
war_merged1 = war_merged.iloc[:,[0,1,6,7,8]]
war_merged1


,Borough,Inhabitants,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Mokotów,217577,Park,Café,Gym
1,Praga-Południe,178726,Café,Restaurant,Italian Restaurant
2,Ursynów,150273,Convenience Store,Chinese Restaurant,Supermarket
3,Wola,140111,Café,Pizza Place,Park
4,Bielany,132026,Café,Tram Station,Supermarket
5,Targówek,123941,Supermarket,Plaza,Market
6,Bemowo,122210,Supermarket,Café,Dessert Shop
7,Śródmieście,117005,Cocktail Bar,Café,Beer Bar
8,Białołęka,119374,Flower Shop,Asian Restaurant,Outdoors & Recreation
9,Ochota,83081,Café,Park,Italian Restaurant


## And let's check in what neighbourhoods this type of venue is underrepresented

In [122]:
war_merged2 = war_merged1.loc[war_merged1['1st Most Common Venue'] !='Café']
war_merged3 = war_merged2.loc[war_merged1['2nd Most Common Venue'] !='Café']
war_merged4 = war_merged3.loc[war_merged1['3rd Most Common Venue'] !='Café']
war_merged4

,Borough,Inhabitants,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
2,Ursynów,150273,Convenience Store,Chinese Restaurant,Supermarket
5,Targówek,123941,Supermarket,Plaza,Market
8,Białołęka,119374,Flower Shop,Asian Restaurant,Outdoors & Recreation
10,Wawer,75991,Hotel,Bus Station,Bakery
12,Ursus,59261,Supermarket,Italian Restaurant,Hotel
14,Włochy,41929,Supermarket,Hotel,Turkish Restaurant
15,Wilanów,35511,Restaurant,Golf Course,Gym / Fitness Center
16,Wesoła,25106,Train Station,Forest,Neighborhood


### This gives me the list of Boroughs where I can enter with the most popular venue type